In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Conv1D, MaxPooling1D, TimeDistributed
from tensorflow.keras.utils import to_categorical

In [2]:
#--- Hằng số dữ liệu ---
OUTPUT_CSV = './Outputs/training-feature.csv'

#--- Hằng số mô hình chuỗi thời gian ---
SEQUENCE_LENGTH = 30    # Số lượng frame liên tiếp được coi là 1 mẫu huấn luyện (ví dụ: 1 giây @ 30 FPS)
NUM_FEATURES = 9        # Số đặc trưng đầu vào (EAR, MAR, PITCH, YAW, ..., UNRELIABLE_POSE)
NUM_CLASSES = 4         # Số trạng thái đầu ra (0: awake, 1: sleep, 2: yawning, 3: passed_out)

In [3]:
#--- Tải và Chuẩn hóa Dữ liệu ---

# Tải file .csv
df = pd.read_csv(OUTPUT_CSV)

# 1. Tách Đặc trưng (X) và Nhãn (y)
X = df[['EAR', 'MAR', 'PITCH', 'YAW', 'ROLL', 'D_SLUMP', 'R_TILT', 'EYE_CL', 'UNRELIABLE_POSE']].values
y = df[['Label']].values

# 2. Chuẩn hoá (Scaling) đặc trưng
# Đảm bảo các đặc trưng nằm trong khoảng từ 0 đến 1 (hoặc -1 đến 1)
scaler = MinMaxScaler(feature_range=(-1,1))
X_scaled = scaler.fit_transform(X)

# 3. Mã hoá Nhãn (One-hot Encoding)
# Chuyển nhãn số (0, 1, 2, 3) thành vector (ví dụ: 2 -> [0, 0, 1, 0])

y_encoded = to_categorical(y, num_classes=NUM_CLASSES)

print(f"Total frames has been scaled: {X_scaled.shape[0]}")

Total frames has been scaled: 2635


In [ ]:
#--- Tạo Chuỗi Thời gian (Sliding Window) ---
"""
Đây là bước quan trọng nhất. Chúng ta cần chuyển mảng 2D (Frame x Features) 
thành mảng 3D (Sample x Time Steps x Features) để LSTM có thể xử lý.
"""
def create_sequences(X, y, seq_length):
    X_seq, y_seq = [] , []

    # Lặp qua tất cả các frame, trừ seq_length frame cuối cùng
    for i in range(len(X) - seq_length + 1):
        # Lấy một cửa sổ (window) X dài seq_length frame
        window_X = X[i : i + seq_length]

        # Lấy nhãn y cho frame cuối cùng của cửa sổ (Nhãn cuối cho chuỗi đó)
        window_y = y_encoded[i + seq_length - 1]

        X_seq.append(window_X)
        y_seq.append(window_y)

    return np.array(X_seq), np.array(y_seq)

# Tạo dữ liệu chuỗi
X_sequences, y_sequences = create_sequences(X_scaled, y, SEQUENCE_LENGTH)

print(f"Final training data has shape: {X_sequences.shape}") # Vd: (2000, 30, 9)
print(f"Label output has shape: {y_sequences.shape}") #Vd (2000, 4)

# Tách tập Train và Test (80% Train, 20% Test)
X_train, X_test, y_train, y_test = train_test_split(
    X_sequences, y_sequences, test_size=0.2, random_state=42, shuffle=True
)
print(f"Shape X_train: {X_train.shape}")

NameError: name 'create_sequences' is not defined

In [ ]:
#--- Xây dựng và Huấn luyện Mô hình CNN-LSTM ---
"""
Kiến trúc này kết hợp CNN để học các mẫu ngắn hạn trong chuỗi (ví dụ: nháy mắt)
và LSTM để học sự phụ thuộc dài hạn (ví dụ: gục đầu kéo dài).
LSTM : Long Short-Term Memory
Dense: Fully Connected

LSTM là một loại Recurrent Neural Network (RNN) đặc biệt được thiết kế để giải quyết vấn đề quên thông tin trong các chuỗi dài.
Nó thực hiện điều này thông qua ba Gate (cổng): 
    Forget Gate: Quyết định thông tin nào từ quá khứ nên được loại bỏ.
    Input Gate: Quyết định thông tin mới nào nên được thêm vào trạng thái hiện tại.
    Output Gate: Quyết định thông tin nào nên được xuất ra làm đầu ra của neuron đó.

tanh, relu là các hàm kích hoạt (activation function) được áp dụng cho đầu ra của mỗi neuron
để đưa ra các quyết định phi tuyến tính (non-linear decisions)
"""

def build_cnn_lstm_model(input_shape, num_classes):
    model = Sequential()

    # 1. Lớp CNN 1D (Học các mẫu cục bộ trên trục thời gian)
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape))
    model.add(MaxPooling1D(pool_size=2))

    # 2. Lớp LSTM (Học sự phụ thuộc chuỗi dài hạn)
    # return_sequences=False vì đây là lớp cuối cùng trước Dense
    # Nếu muốn thêm lớp LSTM nữa, phải đặt return_sequences=True cho lớp trước đó
    model.add(LSTM(128,activation='tanh')) # Sử dụng 128 đơn vị ghi nhớ để học mối quan hệ giữa các frame theo thời gian, sử dụng tanh để xử lý dữ liệu chuỗi hiệu quả.
    model.add(Dropout(0.5)) # Ngăn chặn Overfitting

    # 3. Lớp Dense (Phân loại)
    model.add(Dense(64, activation='relu')) #Sử dụng 64 neuron để xử lý trung gian, sử dụng relu để đảm bảo tính toán nhanh và khả năng học phi tuyến tính mạnh mẽ.
    model.add(Dropout(0.3))
    model.add(Dense(num_classes, activation = 'softmax')) # Softmax cho phân loại đa lớp

    return model

# Xây dựng mô hình
model = build_cnn_lstm_model((SEQUENCE_LENGTH, NUM_FEATURES), NUM_CLASSES)

# Biên dịch mô hình
model.compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)

model.summary()


In [ ]:
#--- Huấn luyện mô hình (có thể điều chỉnh epochs và batch_size) ---
history = model.fit(
    X_train,
    y_train,
    epochs = 50,       # Số lần học trên toàn bộ dữ liệu
    batch_size = 32,   # Số lượng mẫu xử lý mỗi lần cập nhật
    validation_data = (X_test, y_test),
    verbose = 1
)

# Đánh giá mô hình cuối cùng
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"\n---Evaluate on Test ---")
print(f"Loss: {loss:.4f}")
print(f"Accuracy: {accuracy * 100:.2f}%")

# Lưu mô hình
model.save('./Outputs/drowsiness_model.h5')
print(f"Model has been saved successfully at: ./Outputs/drowsiness_model.h5")

print(f"\n--- DETAILED MODEL PERFORMANCE ANALYSIS ---")

# 1. Dự đoán xác suất trên tập test
y_pred_probs = model.predict(X_test)

# 2. Chuyển đổi từ One-hot Encoding (Dự đoán và Thực tế) sang nhãn số
# Lấy chỉ mục các xác suất lớn nhất
y_pred_labels = np.argmax(y_pred_probs, axis = 1)
y_true_labels = np.argmax(y_test, axis = 1)

# 3. In báo cáo Phân loại (Classification Report)
target_names = ['0: awake', '1: sleep', '2: yawning', '3: passed_out']
print(f"\n[Classification Report]")
print(classification_report(y_true_labels, y_pred_labels,target_names=target_names, digits=4))

# 4. In Ma trận nhầm lẫn (Confusion Matrix)
cm = confusion_matrix(y_true_labels, y_pred_labels)
print(f"\n[Confusion Matrix]")
print(cm)